In [ ]:
# For IceCube Data analysis .... 
# This is for Data analysis the example basic is for Sensitivity 


In [1]:
# imports
from constants import pdm_constants as const
from config import config
from pone_aeff import Aeff
from dm2nu import DM2Nu
from atm_shower import Atm_Shower
from detectors import Detector
import numpy as np
import matplotlib.pyplot as plt
from limit_calc import Limits
from bkgrd_calc import Background
from signal_calc import Signal
from pdm import PDM
from scipy.interpolate import UnivariateSpline
from scipy.stats import chi2
from scipy.stats import *
from pone_aeff import Aeff
import pickle
import csv
from matplotlib import rc

In [2]:
# picture path
PICS = '../pics/'
name=[r'$\nu_{\mu}$', r"$\nu_e$", r"$\nu_{\tau}$"]  # ,r'$\nu_{e}$',r'$\nu_{\tau}$'
# Plotting standards
std_size = 6.  # Size of the plot
fontsize = 15.  # Fontsize in the plot
lw=0.5  # Linewidth
h_length=0.5  # Handle length for the legends
export_dpi = 500  # Dpi for the image export
color_pone='#7570b3'
color_ice='#e7298a'
color_combined='#33a02c'
color_thermal='#1f78b4'
color_unitary='k'
color_4='#1b9e77'
color_6='#d95f02'
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

In [24]:
surface_fluxe = pickle.load(open("../data/" +
                                                   "surf_store_v1.p", "rb"))

In [26]:
e_grid = surface_fluxe[0][0]

In [3]:
def ice_parser(filename):
    
        store = []
        with open(filename, newline='') as csvfile:
            reader = csv.reader(csvfile)
            for row_num, row in enumerate(reader):
                if row_num == 0:
                    continue
                store.append(row[0].split())
        store = np.array(store, dtype=float)

        return store

In [4]:
bkgrd_ice_data = [
                        '../data/icecube_10year_ps/events/IC40_exp.csv',
                        '../data/icecube_10year_ps/events/IC59_exp.csv',
                        '../data/icecube_10year_ps/events/IC79_exp.csv',
                        '../data/icecube_10year_ps/events/IC86_I_exp.csv',
                        '../data/icecube_10year_ps/events/IC86_II_exp.csv',
                        '../data/icecube_10year_ps/events/IC86_III_exp.csv',
                        '../data/icecube_10year_ps/events/IC86_IV_exp.csv',
                        '../data/icecube_10year_ps/events/IC86_V_exp.csv',
                        '../data/icecube_10year_ps/events/IC86_VI_exp.csv',
                        '../data/icecube_10year_ps/events/IC86_VII_exp.csv',
                    ]
bkgrd_ice_dic = {
                        0: ice_parser(bkgrd_ice_data[0]),
                        1: ice_parser(bkgrd_ice_data[1]),
                        2: ice_parser(bkgrd_ice_data[2]),
                        3: ice_parser(bkgrd_ice_data[3]),
                        4: ice_parser(bkgrd_ice_data[4]),
                        5: ice_parser(bkgrd_ice_data[4]),
                        6: ice_parser(bkgrd_ice_data[4]),
                        7: ice_parser(bkgrd_ice_data[4]),
                        8: ice_parser(bkgrd_ice_data[4]),
                        9: ice_parser(bkgrd_ice_data[4]),
                    }

In [37]:
ice_data_dic = {}
for i in bkgrd_ice_dic.keys():
    tmp_hist_data, bins_edge = np.histogram(bkgrd_ice_dic[i][:,1], bins=np.log10(e_grid) )
    ice_data_dic[i] = tmp_hist_data

In [39]:
ice_data_dic[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    1,    5,   11,   64,  242,  809, 1544, 2147, 2398,
       1928, 1854, 1235, 1006,  790,  601,  607,  408,  394,  337,  221,
        258,  281,  319,  421,  366,  524,  693,  848, 1129, 1284, 1825,
       2626, 3286, 3190, 1540,  887,  430,  199,   89,   39,   33,   18,
          5,    5,    1,    2,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [32]:
bins_edge

array([8.91250938e-02, 1.12201845e-01, 1.41253754e-01, 1.77827941e-01,
       2.23872114e-01, 2.81838293e-01, 3.54813389e-01, 4.46683592e-01,
       5.62341325e-01, 7.07945784e-01, 8.91250938e-01, 1.12201845e+00,
       1.41253754e+00, 1.77827941e+00, 2.23872114e+00, 2.81838293e+00,
       3.54813389e+00, 4.46683592e+00, 5.62341325e+00, 7.07945784e+00,
       8.91250938e+00, 1.12201845e+01, 1.41253754e+01, 1.77827941e+01,
       2.23872114e+01, 2.81838293e+01, 3.54813389e+01, 4.46683592e+01,
       5.62341325e+01, 7.07945784e+01, 8.91250938e+01, 1.12201845e+02,
       1.41253754e+02, 1.77827941e+02, 2.23872114e+02, 2.81838293e+02,
       3.54813389e+02, 4.46683592e+02, 5.62341325e+02, 7.07945784e+02,
       8.91250938e+02, 1.12201845e+03, 1.41253754e+03, 1.77827941e+03,
       2.23872114e+03, 2.81838293e+03, 3.54813389e+03, 4.46683592e+03,
       5.62341325e+03, 7.07945784e+03, 8.91250938e+03, 1.12201845e+04,
       1.41253754e+04, 1.77827941e+04, 2.23872114e+04, 2.81838293e+04,
      

In [7]:
bkgrd_total = np.array([])
for i in bkgrd_ice_dic:
    bkgrd_total = np.add(bkgrd_total, np.array(bkgrd_ice_dic[i]))

ValueError: operands could not be broadcast together with shapes (0,) (36900,7) 